In [1]:
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('qnli1.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)

df2 = pd.read_csv('qnli1.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)


C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df1.head()

,question,sentence,label
0,When did the third Digimon series begin?,Unlike the two seasons before it and most of t...,not_entailment
1,Which missile batteries often have individual ...,"When MANPADS is operated by specialists, batte...",not_entailment
2,What two things does Popper argue Tarski's the...,He bases this interpretation on the fact that ...,entailment
3,What is the name of the village 9 miles north ...,"On 31 December 1853, the Ottoman forces at Cal...",entailment
4,What famous palace is located in London?,London contains four World Heritage Sites: the...,not_entailment


In [4]:
df1.dropna
df2.dropna

<bound method DataFrame.dropna of                                                question  \
0              When did the third Digimon series begin?   
1     Which missile batteries often have individual ...   
2     What two things does Popper argue Tarski's the...   
3     What is the name of the village 9 miles north ...   
4              What famous palace is located in London?   
...                                                 ...   
1427  The first American to travel into space was whom?   
1428  How many years does a president have in office...   
1429     What is the second principle of Protestantism?   
1430  WHat is the earliest surviving use of the clan...   
1431                                What are morphemes?   

                                               sentence           label  
0     Unlike the two seasons before it and most of t...  not_entailment  
1     When MANPADS is operated by specialists, batte...  not_entailment  
2     He bases this interpretation 

In [5]:
df1["label"].value_counts()

entailment        759
not_entailment    673
Name: label, dtype: int64

In [6]:
df3 = df1[df1['label'] == 'entailment']

In [7]:
# randomly picking 250 rows
df3 = df3.sample(n=250, replace=True)

In [8]:
df3["label"].value_counts()

entailment    250
Name: label, dtype: int64

In [9]:
df4 = df2[df2['label'] == 'not_entailment']

In [10]:
# randomly picking 250 rows
df4 = df4.sample(n=250, replace=True)

In [11]:
df4["label"].value_counts()

not_entailment    250
Name: label, dtype: int64

In [12]:
#concating acceptable and unacceptable
frames = [df3, df4]

temp = pd.concat(frames)

In [13]:
temp["label"].value_counts()

entailment        250
not_entailment    250
Name: label, dtype: int64

In [14]:
text1 = temp['question']+temp['sentence']
emotion= temp['label'].tolist()

In [15]:
text=text1.values.tolist()

In [16]:
temp["label"].value_counts()

entailment        250
not_entailment    250
Name: label, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2, random_state=42)

In [18]:
import ktrain
from ktrain import text
MODEL_NAME = 'prajjwal1/bert-tiny'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [19]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 35
	95percentile : 58
	99percentile : 74


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 36
	95percentile : 58
	99percentile : 72


In [20]:
model = t.get_classifier()

C:\Users\moshi\anaconda3\lib\site-packages\ktrain\text\preprocessor.py:1069: UserWarning: Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn('Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. ' +\


In [21]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [22]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
67/67 [==============================] - 29s 330ms/step - loss: 0.6994 - accuracy: 0.5050 - val_loss: 0.7094 - val_accuracy: 0.4600
Epoch 2/5
67/67 [==============================] - 23s 346ms/step - loss: 0.6942 - accuracy: 0.5225 - val_loss: 0.7012 - val_accuracy: 0.4700
Epoch 3/5
67/67 [==============================] - 23s 346ms/step - loss: 0.6853 - accuracy: 0.5425 - val_loss: 0.6946 - val_accuracy: 0.5200
Epoch 4/5
67/67 [==============================] - 24s 358ms/step - loss: 0.6838 - accuracy: 0.5525 - val_loss: 0.6930 - val_accuracy: 0.5300
Epoch 5/5
67/67 [==============================] - 23s 338ms/step - loss: 0.6797 - accuracy: 0.5800 - val_loss: 0.6935 - val_accuracy: 0.4800


In [23]:
class_names=["TRUE","FALSE"]

In [24]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

        TRUE       0.46      0.78      0.58        46
       FALSE       0.55      0.22      0.32        54

    accuracy                           0.48       100
   macro avg       0.50      0.50      0.45       100
weighted avg       0.51      0.48      0.44       100



array([[36, 10],
       [42, 12]], dtype=int64)